<a href="https://colab.research.google.com/github/wairiukoirwine/ML11/blob/main/full_proof_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import io
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, f1_score
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

# ===================== UPLOAD BUTTON =====================
upload_widget = widgets.FileUpload(accept='.csv', multiple=False)
display(upload_widget)

# Dropdowns for features, target, model
feature_select = widgets.SelectMultiple(options=[], description='Features')
target_select = widgets.Dropdown(options=[], description='Target')
model_select = widgets.Dropdown(options=['linear', 'tree'], description='Model')
run_button = widgets.Button(description='Run ML Pipeline')
output = widgets.Output()
display(feature_select, target_select, model_select, run_button, output)

# ===================== HELPER FUNCTIONS =====================
def load_uploaded_file(widget):
    key = list(widget.value.keys())[0]
    content = widget.value[key]['content']
    df = pd.read_csv(io.BytesIO(content))
    return df

def infer_problem_type(y_series):
    if pd.api.types.is_numeric_dtype(y_series):
        return 'regression' if y_series.nunique() > 5 else 'classification'
    else:
        return 'classification'

def build_preprocessor(X):
    numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
    categorical_cols = X.select_dtypes(exclude=[np.number]).columns.tolist()

    numeric_transformer = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])

    preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

    return preprocessor

# ===================== TRAIN AND EVALUATE =====================
def train_and_evaluate(df, feature_cols, target_col, model_choice='linear'):
    X = df[feature_cols]
    y = df[target_col]
    problem_type = infer_problem_type(y)
    print(f"Detected problem type: {problem_type}")

    preprocessor = build_preprocessor(X)

    if model_choice.lower() == 'linear':
        model = LinearRegression()
    else:
        model = DecisionTreeRegressor() if problem_type == 'regression' else DecisionTreeClassifier()

    pipeline = Pipeline([('preprocessor', preprocessor), ('model', model)])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)

    print("\n=== MODEL EVALUATION ===")
    if problem_type == 'regression':
        print(f"MSE: {mean_squared_error(y_test, y_pred)}")
        print(f"R2: {r2_score(y_test, y_pred)}")
    else:
        print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
        print(f"F1 Score: {f1_score(y_test, y_pred, average='weighted')}")

    # Save predictions
    pred_df = X_test.copy()
    pred_df['Actual'] = y_test.values
    pred_df['Predicted'] = y_pred
    pred_df.to_csv("predictions.csv", index=False)
    print("Predictions saved as predictions.csv")

    # ===================== PREDICTION BAR GRAPH BY FIRST FEATURE =====================
    plt.figure(figsize=(10,5))
    first_feature = feature_cols[0]
    unique_values = np.sort(X_test[first_feature].unique())
    actual_means = [y_test[X_test[first_feature] == val].mean() for val in unique_values]
    predicted_means = [y_pred[X_test[first_feature] == val].mean() for val in unique_values]

    width = 0.4
    plt.bar(np.arange(len(unique_values)) - width/2, actual_means, width=width, label='Actual', color='blue')
    plt.bar(np.arange(len(unique_values)) + width/2, predicted_means, width=width, label='Predicted', color='red')
    plt.xticks(np.arange(len(unique_values)), unique_values)
    plt.xlabel(first_feature)
    plt.ylabel(target_col)
    plt.title('Actual vs Predicted by Feature')
    plt.legend()
    plt.show()

# ===================== CALLBACKS =====================
def on_upload_change(change):
    with output:
        clear_output()
        global df
        df = load_uploaded_file(upload_widget)
        print("File loaded. Select features, target, and model, then click 'Run ML Pipeline'.")
        options = df.columns.tolist()
        feature_select.options = options
        target_select.options = options

upload_widget.observe(on_upload_change, names='value')

def on_run_click(b):
    with output:
        clear_output()
        feature_cols = list(feature_select.value)
        target_col = target_select.value
        model_choice = model_select.value
        if not feature_cols or not target_col:
            print("Please select at least one feature and a target column.")
            return
        train_and_evaluate(df, feature_cols, target_col, model_choice)

run_button.on_click(on_run_click)


FileUpload(value={}, accept='.csv', description='Upload')

SelectMultiple(description='Features', options=(), value=())

Dropdown(description='Target', options=(), value=None)

Dropdown(description='Model', options=('linear', 'tree'), value='linear')

Button(description='Run ML Pipeline', style=ButtonStyle())

Output()